<a href="https://colab.research.google.com/github/VirtueZhao/Dive_into_Deep_Learning/blob/main/5_5_GPUs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Dec 10 03:25:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:0')

(device(type='cpu'),
 <torch.cuda.device at 0x7f4b2622c210>)

In [7]:
torch.cuda.device_count()

1

In [9]:
def try_gpu(i=0):
  if torch.cuda.device_count() >= i + 1:
    return torch.device(f'cuda:{i}')
  return torch.device('cpu')

def try_all_gpus():
  devices = [torch.device(f'cuda:{i}')
        for i in range(torch.cuda.device_count())]
  return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [10]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [14]:
X = torch.ones(2, 3, device=try_gpu())
print(X.device)
print(X)

Y = torch.rand(2, 3, device=try_gpu(1))
print(Y.device)
print(Y)



cuda:0
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
cpu
tensor([[0.0252, 0.7410, 0.5737],
        [0.3568, 0.7253, 0.5318]])


In [16]:
X + Y

RuntimeError: ignored

In [17]:
Z = Y.cuda(0)
print(Y)
print(Z)

tensor([[0.0252, 0.7410, 0.5737],
        [0.3568, 0.7253, 0.5318]])
tensor([[0.0252, 0.7410, 0.5737],
        [0.3568, 0.7253, 0.5318]], device='cuda:0')


In [18]:
X + Z

tensor([[1.0252, 1.7410, 1.5737],
        [1.3568, 1.7253, 1.5318]], device='cuda:0')

In [21]:
Z.cuda(0) is Z

True

In [22]:
net = nn.Sequential(nn.Linear(3,1))
net = net.to(device=try_gpu())

In [23]:
net(X)

tensor([[-0.5954],
        [-0.5954]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [24]:
net[0].weight.data.device

device(type='cuda', index=0)